In [1]:
import tkinter as tk
from tkinter import messagebox
import cv2
import os
from PIL import Image
import numpy as np
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pickle
import joblib

In [2]:
window = tk.Tk()

window.title("Hệ thống nhận dạng khuôn mặt")

l1 = tk.Label(window,text="MSSV", font=("Roboto", 20))
l1.grid(column=0, row=0)
t1 = tk.Entry(window, width=50, bd=5)
t1.grid(column=1, row=0)

l2 = tk.Label(window,text="Tuổi", font=("Roboto", 20))
l2.grid(column=0, row=1)
t2 = tk.Entry(window, width=50, bd=5)
t2.grid(column=1, row=1)

l3 = tk.Label(window,text="Họ và tên", font=("Roboto", 20))
l3.grid(column=0, row=2)
t3 = tk.Entry(window, width=50, bd=5)
t3.grid(column=1, row=2)

l4 = tk.Label(window,text="Ngày sinh", font=("Roboto", 20))
l4.grid(column=0, row=3)
t4 = tk.Entry(window, width=50, bd=5)
t4.grid(column=1, row=3)


def generate_dataset() :
    if t1.get() == "" or t2.get() == "" or t3.get() == "" or t4.get() == "":
        messagebox.showinfo("Result", "Hãy nhập đầy đủ các trường")
    else:
        face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        def face_cropped(img):
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray, 1.3, 5)
            if faces is () :
                return None
            for(x, y, w, h) in faces:
                cropped_face = img[y:y+h, x:x+w]
            return cropped_face
        cap = cv2.VideoCapture(0)
        id = int(t1.get())
        img_id = 0

        while True :
            ret, frame = cap.read()
            if face_cropped(frame) is not None:
                img_id += 1
                face = cv2.resize(face_cropped(frame), (200, 200))
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                file_name_path = "data/user." + str(id) + "-" + t3.get() + "." + str(img_id) + ".jpg"
                cv2.imwrite(file_name_path, face)
                cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

                cv2.imshow("Cropped face", face)
                if cv2.waitKey(1) == 13 or int(img_id) == 100:
                    break
        cap.release()
        cv2.destroyAllWindows()

b1 = tk.Button(window, text="Thêm dataset", font=("Roboto", 20), bg="orange", fg="black", command=generate_dataset)
b1.grid(column=0, row=4)


def train_classifier() :
    data_dir = "C:/Users/Admin/jupyter/face-recognition-PCA/data" 
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []
    
    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        text = os.path.split(image)[1].split(".")[1]
        id = int(text.split("-")[0])
        faces.append(imageNp)
        ids.append(id)
    ids = np.array(ids)
    #feature extraction
    pca = PCA(n_components=1)
    image_pca = []
    for img_vector in faces:
        pca.fit(img_vector)
        img_pca = pca.transform(img_vector)
        image_pca.append(img_pca.flatten())
    #train classifier and save it
    clf = SVC(kernel = 'rbf', class_weight='balanced', probability=True)
    clf.fit(image_pca, ids)
    joblib.dump(clf, 'classifier.pkl')
    messagebox.showinfo("Result", "Training hoàn thành!")

b2 = tk.Button(window, text="Training", font=("Roboto", 20), bg="green", fg="black", command=train_classifier)
b2.grid(column=1, row=4)

def detect_face():
    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf, pca) :
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)

        coords = []
        for(x, y, w, h) in features:
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            img_cropped = cv2.resize(gray_image[y:y+h,x:x+w], (200, 200))
            pca.fit(img_cropped)
            img_pca = pca.transform(img_cropped).flatten()
            pred = clf.predict([img_pca])
            proba = clf.predict_proba([img_pca])
            max_proba = max(proba.flatten())
            print(max_proba)
            if int(max_proba * 100) >= 97:
                cv2.putText(img, str(pred), (x, y-5), cv2.FONT_HERSHEY_COMPLEX, 0.8, color, 1, cv2.LINE_AA)
            else :
                cv2.putText(img, "UNKNOWN", (x, y-5), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)

            coords = [x, y, w, h]
        return coords

    def recognize(img, clf, faceScascade, pca) :
        coords = draw_boundary(img, faceCascade, 1.3, 5, (255, 255, 255), "Face", clf, pca)
        return img

    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    clf = joblib.load("classifier.pkl")
    pca = PCA(n_components=1)
    video_capture = cv2.VideoCapture(0)

    while True:
        ret, img = video_capture.read()
        img = recognize(img, clf, faceCascade, pca)
        cv2.imshow("face detection", img)

        if cv2.waitKey(1) == 13 :
            break
    video_capture.release()
    cv2.destroyAllWindows()

b3 = tk.Button(window, text="Nhận dạng", font=("Roboto", 20), bg="pink", fg="black", command=detect_face)
b3.grid(column=2, row=4)


window.geometry("1000x640")
window.mainloop()

<>:34: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:34: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\Admin\AppData\Local\Temp\ipykernel_22172\361413778.py:34: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is () :


0.9847226156443889
0.9804655158092933
0.9804655158092933
0.9778718372495053
0.9778718372495053
0.9733807654631298
0.9733807654631298
0.9733807654631298
0.9733807654631298
0.9705126114867825
0.9283663420113
0.9283663420113
0.9283663420113
0.9283663420113
0.9737694768730714
0.9737694768730714
0.9737694768730714
0.9737694768730714
0.9655610773779321
0.9655610773779321
0.9655610773779321
0.9655610773779321
0.7639841686488146
0.7639841686488146
0.7639841686488146
0.7639841686488146
0.6781619164246244
0.6781619164246244
0.6781619164246244
0.6781619164246244
0.6009276584825545
0.6009276584825545
0.6009276584825545
0.6009276584825545
0.5910262821495949
0.5910262821495949
0.5910262821495949
0.5910262821495949
0.6579795245738247
0.6579795245738247
0.6579795245738247
0.6579795245738247
0.8030766306843274
0.6822336612562442
0.6822336612562442
0.6822336612562442
0.6822336612562442
0.7847500980847166
0.7847500980847166
0.7847500980847166
0.7847500980847166
0.7239853016718478
0.7239853016718478
0.723

0.9682072120699574
0.9682072120699574
0.9682072120699574
0.9660961609672374
0.9660961609672374
0.9660961609672374
0.9660961609672374
0.9657636088777294
0.9657636088777294
0.9657636088777294
0.9657636088777294
0.96690831599219
0.96690831599219
0.96690831599219
0.96690831599219
0.9661642022775035
0.9661642022775035
0.9661642022775035
0.9661642022775035
0.9685409284878809
0.9685409284878809
0.9685409284878809
0.9685409284878809
0.9648394501590035
0.9700365124421367
0.9700365124421367
0.9700365124421367
0.9700365124421367
0.9704861583496728
0.9704861583496728
0.9704861583496728
0.9704861583496728
0.972018983803204
0.972018983803204
0.972018983803204
0.972018983803204
0.9699555965488781
0.9699555965488781
0.9699555965488781
0.9699555965488781
0.9713741974680359
0.9713741974680359
0.9713741974680359
0.9713741974680359
0.9715916188070879
0.971186053560278
0.971186053560278
0.971186053560278
0.971186053560278
0.9914618547817245
0.9916014047732459
0.9916014047732459
0.9916014047732459
0.9916014